In [1]:
# --- Instalação de Bibliotecas ---
!pip install pandas openpyxl ics PyPDF2 google-generativeai -q

# --- Importações Necessárias ---
import json
import os
from google.colab import drive, files
from google.colab import userdata # Para acessar segredos do Colab
import io
import PyPDF2
import google.generativeai as genai
import pandas as pd
from ics import Calendar, Event
from datetime import datetime, timedelta, time
import re # Para ajudar a extrair JSON da resposta da LLM
import textwrap # NOVA IMPORTAÇÃO

# --- Módulo 0: Gerenciador de Dados do Aluno ---
DRIVE_MOUNTED = False
DRIVE_BASE_PATH = '/content/drive/MyDrive/DadosAlunosColab/' # Caminho padrão no Drive

def mount_drive():
    global DRIVE_MOUNTED, DRIVE_BASE_PATH
    if DRIVE_MOUNTED:
        return
    try:
        drive.mount('/content/drive')
        os.makedirs(DRIVE_BASE_PATH, exist_ok=True)
        DRIVE_MOUNTED = True
        print("Google Drive montado com sucesso.")
    except Exception as e:
        print(f"Erro ao montar o Google Drive: {e}. Usando armazenamento local para esta sessão.")
        DRIVE_BASE_PATH = './DadosAlunosColab_local/'
        os.makedirs(DRIVE_BASE_PATH, exist_ok=True)
        DRIVE_MOUNTED = False

def get_student_data_path(student_id):
    return os.path.join(DRIVE_BASE_PATH, f'{student_id}.json')

def save_student_data(student_id, data):
    file_path = get_student_data_path(student_id)
    try:
        with open(file_path, 'w', encoding='utf-8') as f:
            json.dump(data, f, indent=4, ensure_ascii=False)
        print(f"Dados do aluno {student_id} salvos em {file_path}")
    except Exception as e:
        print(f"Erro ao salvar dados do aluno {student_id}: {e}")

def load_student_data(student_id):
    file_path = get_student_data_path(student_id)
    if os.path.exists(file_path):
        try:
            with open(file_path, 'r', encoding='utf-8') as f:
                data = json.load(f)
            return data
        except Exception as e:
            print(f"Erro ao carregar dados do aluno {student_id}: {e}")
            return {}
    else:
        return {}

# --- Módulo 1: Agente de Currículo (BNCC) ---
bncc_topics_data = {
    'Ensino Médio': {
        '1ª Série': {
            'Matemática': [
                {'codigo': 'EM13MAT101', 'descricao': 'Interpretar criticamente situações econômicas, sociais e das Ciências da Natureza que envolvam a variação de grandezas...'},
                {'codigo': 'EM13MAT102', 'descricao': 'Analisar tabelas, gráficos e informações para tomar decisões...'},
            ],
            'Língua Portuguesa': [
                {'codigo': 'EM13LP01', 'descricao': 'Relacionar o texto, tanto na produção como na leitura/escuta, com suas condições de produção e seu contexto sócio-histórico...'},
            ]
        },
        '2ª Série': {
            'História': [
                {'codigo': 'EM13CHS101', 'descricao': 'Analisar e comparar diferentes narrativas expressas em diversas linguagens...'},
                {'codigo': 'EM13CHS102', 'descricao': 'Identificar, analisar e discutir as circunstâncias históricas, geográficas, políticas, econômicas, sociais, ambientais e culturais de matrizes conceituais...'}
            ],
            'Língua Portuguesa': [
                {'codigo': 'EM13LP02', 'descricao': 'Analisar modos de organização da sociedade contemporânea...'},
                {'codigo': 'EM13LP03', 'descricao': 'Analisar criticamente discursos e práticas sociais, considerando diferentes pontos de vista...'}
            ],
            'Matemática': [
                {'codigo': 'EM13MAT201', 'descricao': 'Propor ou participar de ações para investigar desafios do mundo contemporâneo...'},
                {'codigo': 'EM13MAT202', 'descricao': 'Planejar e executar pesquisa amostral sobre questões relevantes...'}
            ],
            'Química': [
                 {'codigo': 'EM13CNT202', 'descricao': 'Analisar as diversas formas de manifestação da vida em seus diferentes níveis de organização...'},
                 {'codigo': 'EM13CNT203', 'descricao': 'Avaliar e prever efeitos de intervenções nos ecossistemas...'}
            ]
        },
        '3ª Série': {
             # Adicionar tópicos para a 3ª série conforme necessário
        }
    }
}

def get_bncc_topics(serie_aluno, disciplina=None):
    nivel_ensino = 'Ensino Médio'
    if nivel_ensino in bncc_topics_data and serie_aluno in bncc_topics_data[nivel_ensino]:
        serie_data = bncc_topics_data[nivel_ensino][serie_aluno]
        if disciplina:
            # Normalizar nome da disciplina para busca
            disciplina_norm = disciplina.strip().title()
            return serie_data.get(disciplina_norm, [])
        else:
            all_topics = []
            for disc_topics in serie_data.values():
                all_topics.extend(disc_topics)
            return all_topics
    return []

# --- Configuração da API do Google Gemini ---
GOOGLE_API_KEY = None
GEMINI_MODEL = None
SAFETY_SETTINGS = [ # Configurações de segurança mais permissivas
    {"category": "HARM_CATEGORY_HARASSMENT", "threshold": "BLOCK_NONE"},
    {"category": "HARM_CATEGORY_HATE_SPEECH", "threshold": "BLOCK_NONE"},
    {"category": "HARM_CATEGORY_SEXUALLY_EXPLICIT", "threshold": "BLOCK_NONE"},
    {"category": "HARM_CATEGORY_DANGEROUS_CONTENT", "threshold": "BLOCK_NONE"},
]
try:
    GOOGLE_API_KEY = userdata.get('GOOGLE_API_KEY')
    if GOOGLE_API_KEY:
        genai.configure(api_key=GOOGLE_API_KEY)
        GEMINI_MODEL = genai.GenerativeModel('gemini-1.5-flash-latest', safety_settings=SAFETY_SETTINGS)
        print("API do Google Gemini configurada com sucesso.")
    else:
        print("Chave de API do Google (GOOGLE_API_KEY) está vazia nos Segredos do Colab. A opção de análise de PDF via LLM estará desabilitada.")
        GOOGLE_API_KEY = None
except userdata.SecretNotFoundError:
    print("Chave de API do Google (GOOGLE_API_KEY) não encontrada nos Segredos do Colab. A opção de análise de PDF via LLM estará desabilitada.")
    GOOGLE_API_KEY = None
except Exception as e:
    print(f"Erro ao configurar a API do Google Gemini: {e}. Verifique se os segredos estão habilitados e a chave está correta. A opção de análise de PDF via LLM estará desabilitada.")
    GOOGLE_API_KEY = None


# --- Módulo 2: Interação Inicial e Coleta de Dados Acadêmicos ---

def collect_initial_info():
    print("🎓 Bem-vindo(a) ao Mentor Educacional Personalizado! 😊")
    student_id_input = input("Para começarmos, qual é o seu primeiro nome (usaremos como identificador)? ").strip()
    student_id = student_id_input if student_id_input else "aluno_padrao"
    if not student_id_input: print(f"Usando identificador padrão: {student_id}")

    student_data = load_student_data(student_id)

    if not student_data:
        print(f"\nOlá, {student_id}! Parece que é nossa primeira conversa.")
        student_data['id'] = student_id
        student_data['nome'] = student_id
        while True:
            serie = input("Qual série do Ensino Médio você está cursando (1ª Série, 2ª Série ou 3ª Série)? ").strip().title()
            if serie in ["1ª Série", "2ª Série", "3ª Série"]:
                student_data['serie'] = serie
                break
            else:
                print("Série inválida. Por favor, use '1ª Série', '2ª Série' ou '3ª Série'.")
        student_data['disciplinas'] = {}
        print(f"Ótimo, {student_data['nome']} da {student_data['serie']}!")
    else:
        print(f"\nOlá de volta, {student_data.get('nome', student_id)}!")
        print(f"Série registrada: {student_data.get('serie', 'Não informada')}")
        confirm_serie = input("A série está correta? (S/N): ").strip().lower()
        if confirm_serie != 's':
            while True:
                serie = input("Qual a série correta (1ª Série, 2ª Série ou 3ª Série)? ").strip().title()
                if serie in ["1ª Série", "2ª Série", "3ª Série"]:
                    student_data['serie'] = serie
                    break
                else:
                    print("Série inválida.")

    save_student_data(student_id, student_data)
    return student_id, student_data

def extract_json_from_llm_response(llm_text_response):
    match = re.search(r"```json\s*([\s\S]*?)\s*```", llm_text_response, re.IGNORECASE)
    json_string = None
    if match:
        json_string = match.group(1)
    else:
        json_start = llm_text_response.find('{')
        json_end = llm_text_response.rfind('}')
        if json_start != -1 and json_end != -1 and json_end > json_start:
            json_string = llm_text_response[json_start : json_end + 1]

    if json_string:
        try:
            return json.loads(json_string)
        except json.JSONDecodeError as e:
            print(f"Falha ao decodificar o JSON extraído da resposta da LLM: {e}")
            print(f"String JSON problemática: {json_string[:500]}...")
            return None
    print("Nenhum JSON válido encontrado na resposta da LLM.")
    return None

def process_pdf_with_llm(pdf_content):
    if not GEMINI_MODEL:
        print("LLM não está configurada. Não é possível processar o PDF.")
        return None
    try:
        reader = PyPDF2.PdfReader(io.BytesIO(pdf_content))
        pdf_text = ""
        if not reader.pages:
            print("Erro: O PDF não contém páginas ou não pôde ser lido.")
            return None
        for page_num in range(len(reader.pages)):
            page = reader.pages[page_num]
            page_text = page.extract_text()
            if page_text:
                pdf_text += page_text + "\n--- Fim da Página ---\n"

        if not pdf_text.strip():
            print("Não foi possível extrair texto do PDF. Pode ser um PDF de imagem ou estar vazio.")
            return None

        print("\nEnviando texto do PDF para análise pela LLM (isso pode levar um momento)...")
        prompt = f"""
        Você é um assistente especialista em extrair dados acadêmicos de boletins escolares do Ensino Médio brasileiro.
        Analise o texto do boletim a seguir e extraia as seguintes informações para CADA disciplina.
        Se uma avaliação se chamar "Prova Final", "PF", "Exame Final" ou similar, identifique-a claramente.
        1.  Nome da Disciplina (ex: "Matemática", "Língua Portuguesa"). Use nomes padronizados e com capitalização de título.
        2.  Média mínima para aprovação na disciplina (se explícito, caso contrário, use 6.0 como padrão).
        3.  Uma lista de avaliações, contendo para cada avaliação:
            a.  "nome": Nome da avaliação (ex: "P1", "Trabalho Bimestral", "Prova Final"). Padronize "Prova Final" se aplicável.
            b.  "peso": Peso percentual da avaliação (ex: 0.3 para 30%). Se não explícito, tente estimar ou deixe como null. A soma dos pesos por disciplina DEVE ser 1.0 (ou 100%). Se os pesos extraídos não somarem 1.0, normalize-os ou indique um problema na extração.
            c.  "nota": Nota obtida pelo aluno (número). Se não realizada ou não informada, deixe como null.
            d.  "faltante": true se a nota for null, false caso contrário.
            e.  "is_pf": true se esta avaliação for a Prova Final/Exame Final, false caso contrário.
        4.  Fórmula de cálculo da média final da disciplina (se explícita, ex: "(P1*0.3)+(TG*0.2)+(PF*0.5)"). Se não, use "ponderada".

        Retorne as informações APENAS no seguinte formato JSON:
        {{
          "disciplinas": {{
            "Nome da Disciplina 1": {{
              "media_minima": 6.0,
              "avaliacoes": [
                {{"nome": "P1", "peso": 0.3, "nota": 7.5, "faltante": false, "is_pf": false}},
                {{"nome": "Trabalho", "peso": 0.2, "nota": 8.0, "faltante": false, "is_pf": false}},
                {{"nome": "Prova Final", "peso": 0.5, "nota": null, "faltante": true, "is_pf": true}}
              ],
              "formula": "(P1*0.3)+(Trabalho*0.2)+(PF*0.5)"
            }}
          }}
        }}
        Se não conseguir extrair informações de forma estruturada, retorne um objeto JSON com "disciplinas": {{}}.
        Texto do Boletim:
        ---
        {pdf_text}
        ---
        """
        response = GEMINI_MODEL.generate_content(prompt)
        print("\nResposta da LLM recebida.")
        extracted_data = extract_json_from_llm_response(response.text)

        if extracted_data and "disciplinas" in extracted_data:
            print("Dados JSON extraídos e analisados com sucesso pela LLM.")
            normalized_disciplinas = {}
            for disc_nome_raw, disc_data in extracted_data["disciplinas"].items():
                disc_nome_norm = disc_nome_raw.strip().title()
                disc_data_normalized_avals = []
                if 'avaliacoes' in disc_data:
                    for aval in disc_data.get('avaliacoes', []):
                        aval['nome'] = aval.get('nome', 'Avaliação Desconhecida').strip().title()
                        aval['is_pf'] = aval.get('is_pf', False)
                        if "prova final" in aval['nome'].lower() or "exame final" in aval['nome'].lower() or "pf" == aval['nome'].lower():
                            aval['is_pf'] = True
                        disc_data_normalized_avals.append(aval)
                    disc_data['avaliacoes'] = disc_data_normalized_avals

                current_sum_pesos = sum(aval.get('peso', 0) or 0 for aval in disc_data.get('avaliacoes', []))
                if current_sum_pesos > 0 and abs(current_sum_pesos - 1.0) > 1e-2:
                    print(f"Aviso LLM: Pesos para '{disc_nome_norm}' somam {current_sum_pesos*100:.1f}%. Isso precisará ser corrigido na validação.")

                disc_data['avaliacoes_faltantes'] = [
                    aval['nome'] for aval in disc_data.get('avaliacoes', []) if (aval.get('faltante', True) or aval.get('nota') is None) and not aval.get('is_pf', False)
                ]
                normalized_disciplinas[disc_nome_norm] = disc_data
            extracted_data["disciplinas"] = normalized_disciplinas
            return extracted_data
        else:
            print("Não foi possível extrair dados estruturados da resposta da LLM no formato esperado.")
            print("Resposta da LLM:", response.text[:500] + "..." if response.text else "Nenhuma resposta")
            return None
    except PyPDF2.errors.PdfReadError as pdf_err:
        print(f"Erro ao ler o arquivo PDF: {pdf_err}. O arquivo pode estar corrompido ou protegido.")
        return None
    except Exception as e:
        print(f"Erro durante o processamento do PDF ou interação com a LLM: {e}")
        return None

def collect_academic_info_manual(student_id, student_data):
    print("\n--- Preenchimento Manual de Informações Acadêmicas ---")
    if 'disciplinas' not in student_data: student_data['disciplinas'] = {}

    while True:
        disciplina_nome = input("Nome da Disciplina (ou 'fim' para terminar): ").strip()
        if disciplina_nome.lower() == 'fim': break

        disciplina_nome_norm = disciplina_nome.title()

        if disciplina_nome_norm not in student_data['disciplinas']:
            student_data['disciplinas'][disciplina_nome_norm] = {'avaliacoes': [], 'formula': 'ponderada', 'media_minima': 6.0}
        disciplina_atual = student_data['disciplinas'][disciplina_nome_norm]

        while True:
            try:
                media_min_str = input(f"  Média mínima para aprovação em {disciplina_nome_norm} (atual: {disciplina_atual.get('media_minima', '6.0')}): ").strip()
                if media_min_str: disciplina_atual['media_minima'] = float(media_min_str)
                elif disciplina_atual.get('media_minima') is None:
                    disciplina_atual['media_minima'] = 6.0
                break
            except ValueError: print("    Entrada inválida.")

        formula_atual = disciplina_atual.get('formula', 'ponderada')
        nova_formula = input(f"  Fórmula de cálculo da média (atual: '{formula_atual}', ex: '(P1*0.3)+(P2*0.5)', 'ponderada'): ").strip()
        if nova_formula: disciplina_atual['formula'] = nova_formula

        print("  Estrutura de avaliação:")
        edit_avals_choice = input(f"  Existem {len(disciplina_atual.get('avaliacoes',[]))} avaliações registradas para '{disciplina_nome_norm}'. Deseja (R)einserir todas, (A)dicionar novas ou (M)anter? (R/A/M): ").strip().lower()

        if edit_avals_choice == 'r':
            disciplina_atual['avaliacoes'] = []

        if edit_avals_choice in ['r', 'a']:
            while True:
                aval_nome_input = input("    Nome da avaliação (ou 'fim' para concluir esta disciplina): ").strip()
                if aval_nome_input.lower() == 'fim': break
                if not aval_nome_input: print("     Nome da avaliação não pode ser vazio."); continue

                aval_nome_norm = aval_nome_input.title()
                is_pf_input = input(f"    '{aval_nome_norm}' é a Prova Final (PF)? (S/N): ").strip().lower()
                is_pf_aval = (is_pf_input == 's')

                aval_peso_percent = -1.0
                while not (0.0 <= aval_peso_percent <= 100.0):
                    try:
                        aval_peso_percent = float(input(f"    Peso percentual de '{aval_nome_norm}' (ex: 30 para 30%): "))
                        if not (0.0 <= aval_peso_percent <= 100.0): print("      Peso deve ser entre 0 e 100.")
                    except ValueError: print("      Entrada inválida para peso.")

                nota_str = input(f"    Nota obtida em '{aval_nome_norm}' (deixe em branco se ainda não feita): ").strip()
                nota, faltante = (None, True)
                if nota_str:
                    try: nota, faltante = (float(nota_str), False)
                    except ValueError: print("      Nota inválida, será considerada como não realizada.")

                existing_aval_idx = next((idx for idx, a in enumerate(disciplina_atual.get('avaliacoes',[])) if a['nome'].title() == aval_nome_norm), -1)
                new_aval_data = {'nome': aval_nome_norm, 'peso': aval_peso_percent / 100.0, 'nota': nota, 'faltante': faltante, 'is_pf': is_pf_aval}

                if existing_aval_idx != -1:
                    if edit_avals_choice == 'r':
                         disciplina_atual['avaliacoes'][existing_aval_idx] = new_aval_data
                         print(f"      Avaliação '{aval_nome_norm}' substituída (modo reinserir).")
                    else:
                         print(f"      Atenção: Avaliação '{aval_nome_norm}' já existe. Se desejava editar, escolha (R)einserir para a disciplina ou edite na etapa de validação.")
                else:
                    disciplina_atual['avaliacoes'].append(new_aval_data)
                    print(f"      Avaliação '{aval_nome_norm}' adicionada.")

        soma_pesos_final_disciplina = sum(aval.get('peso', 0) or 0 for aval in disciplina_atual.get('avaliacoes', []))
        if abs(soma_pesos_final_disciplina - 1.0) > 1e-3 and soma_pesos_final_disciplina > 0:
            print(f"  Atenção: O peso total das avaliações para '{disciplina_nome_norm}' é {soma_pesos_final_disciplina*100:.1f}%. Idealmente, deveria ser 100%.")

        disciplina_atual['avaliacoes_faltantes'] = [a['nome'] for a in disciplina_atual.get('avaliacoes', []) if (a.get('faltante', True) or a.get('nota') is None) and not a.get('is_pf',False)]
        print(f"  Informações de '{disciplina_nome_norm}' atualizadas.")

    save_student_data(student_id, student_data)
    return student_data

def merge_llm_data_with_student_data(student_data_existing, llm_extracted_data, replace_all_disciplines=False):
    if not llm_extracted_data or 'disciplinas' not in llm_extracted_data:
        print("LLM: Nenhum dado de disciplina válido para mesclar.")
        return student_data_existing

    llm_disciplinas = llm_extracted_data['disciplinas'] # Já normalizado em process_pdf_with_llm

    if replace_all_disciplines:
        print("LLM: Substituindo todas as disciplinas existentes pelos dados extraídos do PDF.")
        student_data_existing['disciplinas'] = llm_disciplinas # Atribuição direta
    else:
        if 'disciplinas' not in student_data_existing:
            student_data_existing['disciplinas'] = {}

        for llm_disc_nome, llm_disc_data in llm_disciplinas.items():
            # llm_disc_nome já está normalizado
            matched_disc_name_in_student_data = None
            for s_disc_name_key in student_data_existing['disciplinas'].keys():
                if s_disc_name_key == llm_disc_nome: # Comparação direta de nomes normalizados
                    matched_disc_name_in_student_data = s_disc_name_key
                    break

            if not matched_disc_name_in_student_data:
                student_data_existing['disciplinas'][llm_disc_nome] = llm_disc_data
                print(f"LLM: Adicionando nova disciplina '{llm_disc_nome}'.")
            else:
                print(f"LLM: Mesclando dados para disciplina existente '{matched_disc_name_in_student_data}'.")
                current_disc = student_data_existing['disciplinas'][matched_disc_name_in_student_data]

                if llm_disc_data.get('media_minima') is not None and current_disc.get('media_minima') is None:
                    current_disc['media_minima'] = llm_disc_data['media_minima']
                if llm_disc_data.get('formula', 'ponderada') != 'ponderada' and current_disc.get('formula', 'ponderada') == 'ponderada':
                    current_disc['formula'] = llm_disc_data['formula']

                if 'avaliacoes' not in current_disc: current_disc['avaliacoes'] = []
                current_avals_map = {aval['nome']: aval for aval in current_disc.get('avaliacoes', [])} # Nomes já normalizados

                for llm_aval in llm_disc_data.get('avaliacoes', []):
                    # llm_aval['nome'] já normalizado
                    if llm_aval['nome'] not in current_avals_map:
                        current_disc['avaliacoes'].append(llm_aval)
                        print(f"  LLM: Adicionando nova avaliação '{llm_aval['nome']}' para '{matched_disc_name_in_student_data}'.")
                    else:
                        existing_aval_in_map = current_avals_map[llm_aval['nome']]
                        if existing_aval_in_map.get('faltante', True) and not llm_aval.get('faltante', True) and llm_aval.get('nota') is not None:
                            existing_aval_in_map['nota'] = llm_aval['nota']
                            existing_aval_in_map['faltante'] = False
                            print(f"  LLM: Atualizando nota para avaliação existente '{llm_aval['nome']}'.")
                        if existing_aval_in_map.get('peso') is None and llm_aval.get('peso') is not None:
                            existing_aval_in_map['peso'] = llm_aval.get('peso')
                            print(f"  LLM: Adicionando peso para avaliação existente '{llm_aval['nome']}'.")
                        if existing_aval_in_map.get('is_pf') is None and llm_aval.get('is_pf') is not None:
                             existing_aval_in_map['is_pf'] = llm_aval.get('is_pf')

                current_disc['avaliacoes_faltantes'] = [
                    aval['nome'] for aval in current_disc.get('avaliacoes', []) if (aval.get('faltante', True) or aval.get('nota') is None) and not aval.get('is_pf', False)
                ]
    return student_data_existing


def display_extracted_pdf_data_for_confirmation(llm_extracted_data):
    print("\n--- Dados Extraídos do PDF pela LLM ---")
    if not llm_extracted_data or 'disciplinas' not in llm_extracted_data or not llm_extracted_data['disciplinas']:
        print("Nenhuma disciplina foi extraída do PDF.")
        return False

    for disc_nome, disc_data in llm_extracted_data['disciplinas'].items():
        print(f"\nDisciplina: {disc_nome}") # Já normalizado
        print(f"  Média Mínima: {disc_data.get('media_minima', 'Não especificada')}")
        print(f"  Fórmula: {disc_data.get('formula', 'ponderada')}")
        print("  Avaliações:")
        if disc_data.get('avaliacoes'):
            for aval in disc_data['avaliacoes']:
                nota_str = f"{aval.get('nota'):.2f}" if isinstance(aval.get('nota'), (int, float)) else "N/A"
                peso_str = f"{aval.get('peso')*100:.1f}%" if isinstance(aval.get('peso'), (int, float)) else "N/A"
                is_pf_str = " (Prova Final)" if aval.get('is_pf') else ""
                print(f"    - Nome: {aval.get('nome', 'N/A')}{is_pf_str}, Peso: {peso_str}, Nota: {nota_str}, Faltante: {aval.get('faltante', True)}")
        else:
            print("    Nenhuma avaliação encontrada para esta disciplina.")
    print("--------------------------------------")
    while True:
        confirm = input("Os dados extraídos parecem corretos e você deseja usá-los (substituindo os existentes)? (S/N): ").strip().lower()
        if confirm == 's':
            return True
        elif confirm == 'n':
            return False
        else:
            print("Opção inválida. Digite S ou N.")


def collect_academic_info(student_id, student_data):
    print("\n🗂️ Passo 2 – Coleta/Atualização de Informações Acadêmicas")
    print("Para uma análise precisa, preciso das suas notas e da estrutura de avaliação.")

    data_collected_or_modified_in_this_step = False
    pdf_data_confirmed_and_used = False # Nova flag

    while True:
        print("\nEscolha uma opção:")
        print("  A - Preencher/Editar Manualmente as informações das disciplinas.")
        if GOOGLE_API_KEY and GEMINI_MODEL:
            print("  B - Enviar boletim em PDF (isso SUBSTITUIRÁ as disciplinas atuais pelas do PDF).")
            valid_choices = ['A', 'B', 'F']
        else:
            print("  B - (Opção de PDF indisponível - API LLM não configurada).")
            valid_choices = ['A', 'F']
        print("  F - Finalizar coleta/edição e prosseguir para análise.")

        choice = input("Sua escolha: ").strip().upper()

        if choice == 'F':
            break
        elif choice == 'A':
            student_data = collect_academic_info_manual(student_id, student_data)
            data_collected_or_modified_in_this_step = True
            pdf_data_confirmed_and_used = False
        elif choice == 'B' and GOOGLE_API_KEY and GEMINI_MODEL:
            print("\n--- Envio de Boletim (PDF) ---")
            print("Por favor, faça o upload do seu arquivo PDF.")
            print("AVISO: As informações extraídas deste PDF SUBSTITUIRÃO todas as disciplinas e notas anteriormente registradas, APÓS SUA CONFIRMAÇÃO.")

            try:
                uploaded = files.upload()
                if not uploaded:
                    print("Nenhum arquivo foi enviado.")
                    continue
                filename = list(uploaded.keys())[0]
                print(f'Arquivo "{filename}" enviado.')
                pdf_content = uploaded[filename]
            except Exception as e:
                print(f"Erro durante o upload do arquivo: {e}")
                continue

            llm_extracted_data = process_pdf_with_llm(pdf_content)

            if llm_extracted_data and llm_extracted_data.get("disciplinas"):
                if display_extracted_pdf_data_for_confirmation(llm_extracted_data):
                    student_data = merge_llm_data_with_student_data(student_data, llm_extracted_data, replace_all_disciplines=True)
                    save_student_data(student_id, student_data)
                    print("Dados extraídos pela LLM foram aplicados, substituindo informações anteriores.")
                    data_collected_or_modified_in_this_step = True
                    pdf_data_confirmed_and_used = True
                else:
                    print("Extração do PDF cancelada pelo usuário. Os dados anteriores foram mantidos.")
                    pdf_data_confirmed_and_used = False
            else:
                print("Não foi possível extrair dados do PDF via LLM ou a extração falhou/retornou vazio.")
                pdf_data_confirmed_and_used = False
        elif choice not in valid_choices:
             print("Opção inválida.")
        else:
            print("Opção inválida ou indisponível.")

    if data_collected_or_modified_in_this_step and not pdf_data_confirmed_and_used and student_data.get('disciplinas'):
        print("\nApós a coleta/edição manual, vamos validar os dados para garantir a precisão.")
        student_data = validate_and_complement_data(student_id, student_data)
    elif pdf_data_confirmed_and_used:
        print("\nDados do PDF confirmados. Prosseguindo diretamente para a análise.")
    elif not student_data.get('disciplinas') and data_collected_or_modified_in_this_step:
        print("\nNenhuma disciplina foi inserida ou modificada.")
    elif not student_data.get('disciplinas'):
        print("\nNenhuma disciplina carregada ou inserida para esta sessão.")


    print("Coleta/edição de informações acadêmicas finalizada.")
    return student_data


# --- Módulo 3: Validação e Complementação dos Dados pelo Usuário ---
def validate_and_complement_data(student_id, student_data):
    print("\n📋 Passo de Validação e Complementação dos Dados")
    print(f"Olá, {student_data.get('nome', student_id)}! Vamos revisar e, se necessário, ajustar as informações das suas disciplinas.")

    if 'disciplinas' not in student_data or not student_data['disciplinas']:
        print("Nenhuma disciplina encontrada para validar. Por favor, adicione informações primeiro.")
        return student_data

    disciplinas_a_revisar = list(student_data['disciplinas'].keys())
    for disc_nome_original in disciplinas_a_revisar:
        if disc_nome_original not in student_data['disciplinas']:
            continue

        print(f"\n--- Validando Disciplina: {disc_nome_original} ---")
        disc_atual = student_data['disciplinas'][disc_nome_original]

        media_min_val = disc_atual.get('media_minima')
        while True:
            input_prompt = f"  Média mínima para aprovação (atual: {media_min_val if media_min_val is not None else 'NÃO DEFINIDA (usando 6.0)'}, ex: 6.0): "
            nova_media_min_str = input(input_prompt).strip()
            if not nova_media_min_str and media_min_val is not None: break
            if not nova_media_min_str and media_min_val is None:
                disc_atual['media_minima'] = 6.0
                print("    Usando 6.0 como padrão para média mínima.")
                break
            try:
                disc_atual['media_minima'] = float(nova_media_min_str)
                break
            except ValueError: print("    Valor inválido para média mínima. Use um número (ex: 6.0).")

        formula_val = disc_atual.get('formula', 'ponderada')
        nova_formula = input(f"  Fórmula de cálculo da média (atual: '{formula_val}', ex: '(P1*0.3)+(P2*0.5)', 'ponderada'): ").strip()
        if nova_formula: disc_atual['formula'] = nova_formula

        print("  Avaliações:")
        if 'avaliacoes' not in disc_atual: disc_atual['avaliacoes'] = []

        current_avals_list = disc_atual.get('avaliacoes', [])
        if not isinstance(current_avals_list, list): current_avals_list = []

        for i, aval in enumerate(current_avals_list):
            nota_display = f"{aval.get('nota', 'N/A'):.2f}" if isinstance(aval.get('nota'), float) else str(aval.get('nota', 'N/A'))
            peso_display = f"{(aval.get('peso', 0) or 0)*100:.1f}%"
            is_pf_str = " (Prova Final)" if aval.get('is_pf') else ""
            print(f"    {i+1}. Nome: {aval.get('nome', 'N/A')}{is_pf_str}, Peso: {peso_display}, Nota: {nota_display}, Faltante: {aval.get('faltante', True)}")

        edit_avals = input("  Deseja (R)einserir todas, (A)dicionar novas, (E)ditar uma existente, ou (M)anter as avaliações para esta disciplina? (R/A/E/M): ").strip().lower()

        temp_avals_for_disc = list(disc_atual.get('avaliacoes', []))

        if edit_avals == 'r':
            temp_avals_for_disc = []
            print(f"  Reinserindo avaliações para {disc_nome_original}:")
        elif edit_avals == 'a':
            print(f"  Adicionando novas avaliações para {disc_nome_original}:")
        elif edit_avals == 'e':
            if not temp_avals_for_disc:
                print("    Nenhuma avaliação para editar. Adicione uma primeiro.")
            else:
                while True:
                    try:
                        idx_to_edit_str = input(f"    Qual o número da avaliação para editar (1-{len(temp_avals_for_disc)})? (ou 'fim'): ")
                        if idx_to_edit_str.lower() == 'fim': break
                        idx_to_edit = int(idx_to_edit_str) - 1
                        if 0 <= idx_to_edit < len(temp_avals_for_disc):
                            aval_to_edit = temp_avals_for_disc[idx_to_edit]
                            print(f"    Editando '{aval_to_edit['nome']}':")

                            new_aval_nome = input(f"      Novo nome (atual: {aval_to_edit['nome']}): ").strip().title()
                            if new_aval_nome: aval_to_edit['nome'] = new_aval_nome

                            new_is_pf_input = input(f"      É Prova Final? (atual: {'S' if aval_to_edit.get('is_pf') else 'N'}) (S/N): ").strip().lower()
                            if new_is_pf_input: aval_to_edit['is_pf'] = (new_is_pf_input == 's')

                            while True:
                                try:
                                    new_peso_str = input(f"      Novo peso % (atual: {(aval_to_edit.get('peso',0) or 0)*100:.1f}): ").strip()
                                    if new_peso_str:
                                        new_peso = float(new_peso_str)
                                        if 0.0 <= new_peso <= 100.0:
                                            aval_to_edit['peso'] = new_peso / 100.0
                                            break
                                        else: print("        Peso deve ser entre 0 e 100.")
                                    else: break
                                except ValueError: print("        Valor inválido para peso.")

                            new_nota_str = input(f"      Nova nota (atual: {aval_to_edit.get('nota', 'N/A')}): ").strip()
                            if new_nota_str:
                                try:
                                    aval_to_edit['nota'] = float(new_nota_str)
                                    aval_to_edit['faltante'] = False
                                except ValueError:
                                    aval_to_edit['nota'] = None
                                    aval_to_edit['faltante'] = True
                                    print("        Nota inválida, será 'não realizada'.")
                            elif new_nota_str == "":
                                aval_to_edit['nota'] = None
                                aval_to_edit['faltante'] = True

                            print(f"      Avaliação '{aval_to_edit['nome']}' atualizada.")
                            break
                        else: print(f"    Número inválido. Deve ser entre 1 e {len(temp_avals_for_disc)}.")
                    except ValueError: print("    Entrada inválida para número.")

        if edit_avals in ['r', 'a']:
            while True:
                aval_nome_input = input("    Nome da avaliação (ou 'fim'): ").strip()
                if aval_nome_input.lower() == 'fim': break
                if not aval_nome_input: print("     Nome da avaliação não pode ser vazio."); continue

                aval_nome_norm = aval_nome_input.title()
                is_pf_input = input(f"    '{aval_nome_norm}' é a Prova Final (PF)? (S/N): ").strip().lower()
                is_pf_aval = (is_pf_input == 's')

                aval_peso_percent = -1.0
                while not (0.0 <= aval_peso_percent <= 100.0):
                    try:
                        aval_peso_str = input(f"    Peso percentual de '{aval_nome_norm}' (0-100, ex: 30): ").strip()
                        aval_peso_percent = float(aval_peso_str)
                        if not (0.0 <= aval_peso_percent <= 100.0): print("      Peso deve ser entre 0 e 100.")
                    except ValueError: print("      Valor inválido para peso.")

                nota_str = input(f"    Nota obtida em '{aval_nome_norm}' (deixe em branco se ainda não feita): ").strip()
                nota, faltante = (None, True)
                if nota_str:
                    try: nota, faltante = (float(nota_str), False)
                    except ValueError: print("      Nota inválida, será 'não realizada'.")

                new_aval_data = {'nome': aval_nome_norm, 'peso': aval_peso_percent/100.0, 'nota': nota, 'faltante': faltante, 'is_pf': is_pf_aval}

                existing_aval_index = next((idx for idx, ex_aval in enumerate(temp_avals_for_disc) if ex_aval['nome'].title() == aval_nome_norm), -1)
                if edit_avals == 'a' and existing_aval_index != -1:
                    print(f"      Avaliação '{aval_nome_norm}' já existe. Não foi adicionada. Use (E)ditar ou (R)einserir.")
                else:
                    if existing_aval_index != -1 and edit_avals == 'r':
                        temp_avals_for_disc[existing_aval_index] = new_aval_data
                        print(f"      Avaliação '{aval_nome_norm}' (reinserida/substituída).")
                    elif existing_aval_index != -1 and edit_avals == 'a':
                         print(f"      Avaliação '{aval_nome_norm}' já existe. Use (E)ditar ou (R)einserir.")
                    else:
                        temp_avals_for_disc.append(new_aval_data)
                        print(f"      Avaliação '{aval_nome_norm}' adicionada.")

        disc_atual['avaliacoes'] = temp_avals_for_disc

        final_sum_pesos = sum(aval.get('peso', 0) or 0 for aval in disc_atual.get('avaliacoes', []))
        if abs(final_sum_pesos - 1.0) > 1e-2 and final_sum_pesos > 0:
            print(f"  Atenção Final: O peso total das avaliações para '{disc_nome_original}' é {final_sum_pesos*100:.1f}%. Idealmente, deveria ser 100%.")

        disc_atual['avaliacoes_faltantes'] = [a['nome'] for a in disc_atual.get('avaliacoes', []) if (a.get('faltante', True) or a.get('nota') is None) and not a.get('is_pf', False)]

    save_student_data(student_id, student_data)
    print("\nValidação e complementação concluídas. Dados salvos.")
    return student_data

# --- Módulo 4: Análise Detalhada da Situação Acadêmica (com LLM) ---

def calculate_current_weighted_score_and_weights(avaliacoes):
    soma_nota_peso = 0
    soma_pesos_com_nota = 0
    soma_total_pesos_sem_pf = 0
    peso_pf = 0
    pf_obj = None

    for aval in avaliacoes:
        peso = aval.get('peso', 0) or 0
        if aval.get('is_pf'):
            peso_pf = peso
            pf_obj = aval
        else:
            soma_total_pesos_sem_pf += peso
            if aval.get('nota') is not None and not aval.get('faltante'):
                soma_nota_peso += aval['nota'] * peso
                soma_pesos_com_nota += peso
    return soma_nota_peso, soma_pesos_com_nota, soma_total_pesos_sem_pf, peso_pf, pf_obj

def llm_analyze_discipline_for_pf(discipline_name, discipline_data, student_name="estudante"):
    if not GEMINI_MODEL:
        print("LLM não configurada. Não é possível analisar para PF.")
        return None

    prompt = f"""
    Você é um mentor educacional para estudantes do ensino médio, como um "irmão mais velho" experiente, paciente e compreensivo.
    Seu papel é acolher, orientar e acompanhar o aluno nos estudos.
    O aluno se chama {student_name}.

    Para a disciplina '{discipline_name}', os dados são:
    Média mínima para aprovação: {discipline_data.get('media_minima')}
    Fórmula de cálculo (informativo): {discipline_data.get('formula', 'ponderada')}
    Avaliações: {json.dumps(discipline_data.get('avaliacoes', []), ensure_ascii=False)}

    Tarefa:
    1. Identifique a(s) avaliação(ões) marcada(s) como Prova Final ('is_pf': true). Se houver mais de uma, considere a de maior peso ou a primeira. Se não houver, mas houver avaliações com nota nula e peso, e a soma dos pesos for 1.0, considere a avaliação faltante de maior peso como a PF para este cálculo. Se não houver PF explícita e a soma dos pesos das avaliações existentes for menor que 1.0, assuma que o peso restante (1.0 - soma_pesos_existentes) é para uma Prova Final implícita.
    2. Para CADA avaliação que NÃO SEJA A Prova Final e que esteja com nota nula ('faltante': true, 'nota': null), estime uma nota. Baseie a estimativa na média das notas já obtidas nas outras avaliações (que não sejam PF) desta disciplina. Se não houver notas anteriores, estime como {discipline_data.get('media_minima', 6.0)}.
    3. Calcule a nota EXATA que o aluno precisa tirar na Prova Final para atingir a média mínima de aprovação. Considere as notas já obtidas e as notas ESTIMADAS para as avaliações intermediárias faltantes.
    4. Se a nota necessária na PF for > 10.0 ou < 0.0, indique isso.
    5. **Elabore a 'explicacao_calculo' de forma clara, didática e empática, como se estivesse conversando com {student_name}.** Explique o que as notas estimadas significam e como chegou à nota da PF. Se for impossível, explique com cuidado. Se já estiver aprovado, parabenize!
       Exemplo de tom para explicação: "Olha, {student_name}, pra gente entender como você está em {discipline_name}, eu dei uma olhada nas suas notas. Naquele trabalho que ainda não tem nota, imaginei que você tiraria um X, baseado no seu desempenho até agora. Com isso em mente, pra passar na Prova Final, você precisaria de um Y. Fica tranquilo(a) que a gente vai ver como chegar lá!"

    Retorne a resposta APENAS no formato JSON abaixo. NÃO inclua nenhum texto antes ou depois do JSON.
    {{
      "disciplina": "{discipline_name}",
      "media_minima": {discipline_data.get('media_minima')},
      "avaliacoes_com_estimativas": [
        {{ "nome": "Nome Avaliação 1", "nota_original": 7.5, "nota_estimada": null, "peso": 0.3, "is_pf": false }},
        {{ "nome": "Nome Avaliação Faltante 1", "nota_original": null, "nota_estimada": 6.0, "peso": 0.2, "is_pf": false }},
        {{ "nome": "Prova Final", "nota_original": null, "nota_estimada": null, "peso": 0.5, "is_pf": true }}
      ],
      "nota_necessaria_pf": "X.XX",
      "explicacao_calculo": "Sua explicação empática aqui...",
      "situacao_pf": "Crítica | Atenção Elevada | Atenção | Confortável | Já Aprovado | Já Reprovado"
    }}
    """
    try:
        print(f"  Enviando '{discipline_name}' para análise da LLM (cálculo de PF)...")
        response = GEMINI_MODEL.generate_content(prompt)
        return extract_json_from_llm_response(response.text)
    except Exception as e:
        print(f"  Erro ao chamar LLM para '{discipline_name}': {e}")
        return None

def analyze_academic_situation(student_id, student_data):
    print("\n📊 Passo de Análise Detalhada da Situação Acadêmica (com LLM)")
    analysis_results_llm = {}
    if 'disciplinas' not in student_data or not student_data['disciplinas']:
        print("Nenhuma disciplina para analisar.")
        return analysis_results_llm

    for disc_nome, disc_data in student_data['disciplinas'].items():
        print(f"\n--- Analisando Disciplina: {disc_nome} ---")
        media_minima = disc_data.get('media_minima')
        avaliacoes = disc_data.get('avaliacoes', [])

        if media_minima is None:
            print("  Média mínima não definida. Análise LLM para PF não será realizada.")
            analysis_results_llm[disc_nome] = {
                'disciplina': disc_nome,
                'media_minima': None,
                'nota_necessaria_pf': "N/A",
                'explicacao_calculo': "Média mínima não informada.",
                'situacao_pf': "Indefinida",
                'avaliacoes_com_estimativas': avaliacoes
            }
            continue

        has_pf = any(a.get('is_pf') for a in avaliacoes)
        pf_com_nota = any(a.get('is_pf') and a.get('nota') is not None for a in avaliacoes)
        intermediarias_faltando = any(not a.get('is_pf') and (a.get('nota') is None or a.get('faltante')) for a in avaliacoes)


        if not has_pf:
             print(f"  Disciplina '{disc_nome}' não tem Prova Final (PF) definida. Cálculo de nota para PF não aplicável.")
             soma_nota_peso, soma_pesos_com_nota, _, _, _ = calculate_current_weighted_score_and_weights(avaliacoes)
             media_atual = (soma_nota_peso / soma_pesos_com_nota) if soma_pesos_com_nota > 0 else 0
             situacao = "Aprovado" if media_atual >= media_minima else "Reprovado"
             if any((a.get('nota') is None or a.get('faltante')) for a in avaliacoes):
                 situacao = "Em Andamento (sem PF)"

             analysis_results_llm[disc_nome] = {
                'disciplina': disc_nome,
                'media_minima': media_minima,
                'nota_necessaria_pf': "N/A (Sem PF)",
                'explicacao_calculo': f"Sem PF definida. Média atual com notas lançadas: {media_atual:.2f}",
                'situacao_pf': situacao,
                'avaliacoes_com_estimativas': avaliacoes
             }
             print(f"  Situação: {situacao} (Média atual: {media_atual:.2f})")
             continue


        if pf_com_nota and not intermediarias_faltando:
            print(f"  Todas as notas, incluindo PF, parecem lançadas para '{disc_nome}'. Não é necessário cálculo de PF pela LLM.")
            soma_nota_peso_total = sum( (a.get('nota',0) or 0) * (a.get('peso',0) or 0) for a in avaliacoes if a.get('nota') is not None )
            soma_pesos_total_com_nota = sum(a.get('peso',0) or 0 for a in avaliacoes if a.get('nota') is not None)
            media_final_calculada = (soma_nota_peso_total / soma_pesos_total_com_nota) if soma_pesos_total_com_nota > 0 else 0.0
            situacao = "Aprovado" if media_final_calculada >= media_minima else "Reprovado"

            soma_total_pesos_disciplina = sum(a.get('peso',0) or 0 for a in avaliacoes)
            if abs(soma_total_pesos_disciplina - 1.0) > 0.01 and soma_total_pesos_disciplina > 0 :
                situacao += " (Atenção: Pesos não somam 100% ou nem todas as notas lançadas com peso foram consideradas)"


            analysis_results_llm[disc_nome] = {
                'disciplina': disc_nome,
                'media_minima': media_minima,
                'nota_necessaria_pf': f"Nota PF já lançada ({[a['nota'] for a in avaliacoes if a.get('is_pf') and a.get('nota') is not None][0]:.2f})",
                'explicacao_calculo': f"Todas as notas lançadas. Média final calculada: {media_final_calculada:.2f}",
                'situacao_pf': situacao,
                'avaliacoes_com_estimativas': avaliacoes
            }
            print(f"  Situação: {situacao} (Média final: {media_final_calculada:.2f})")
            continue

        llm_result = llm_analyze_discipline_for_pf(disc_nome, disc_data, student_data.get('nome', 'estudante'))

        if llm_result and "nota_necessaria_pf" in llm_result:
            analysis_results_llm[disc_nome] = llm_result
            print(f"  LLM Análise para '{disc_nome}':")
            print(f"    Média Mínima: {llm_result.get('media_minima')}")
            if llm_result.get('avaliacoes_com_estimativas'):
                print("    Avaliações (com estimativas se aplicável):")
                for aval in llm_result['avaliacoes_com_estimativas']:
                    nota_str = f"{aval.get('nota_original')}"
                    if aval.get('nota_estimada') is not None:
                        nota_str += f" (Estimada pela gente: {aval['nota_estimada']})"
                    elif aval.get('nota_original') is None:
                        nota_str = "Ainda não feita"
                    is_pf_str = " (PF)" if aval.get('is_pf') else ""
                    print(f"      - {aval['nome']}{is_pf_str}: {nota_str}, Peso: {aval.get('peso',0)*100:.0f}%")
            print(f"    Nota Necessária na PF: {llm_result.get('nota_necessaria_pf')}")
            print(f"    Situação PF: {llm_result.get('situacao_pf')}")
            # MODIFICAÇÃO AQUI:
            explicacao_formatada = textwrap.fill(llm_result.get('explicacao_calculo', "Sem explicação da LLM."), width=100, initial_indent="    Mentor diz: \"", subsequent_indent="    ")
            print(explicacao_formatada + "\"") # Adiciona aspas finais
        else:
            print(f"  Falha ao obter análise da LLM para '{disc_nome}'.")
            analysis_results_llm[disc_nome] = {
                'disciplina': disc_nome, 'media_minima': media_minima,
                'nota_necessaria_pf': "Erro LLM", 'explicacao_calculo': "Não consegui calcular direitinho com a IA desta vez, podemos tentar de novo ou verificar os dados.",
                'situacao_pf': "Indefinida (Erro LLM)", 'avaliacoes_com_estimativas': avaliacoes
            }

    if GEMINI_MODEL and analysis_results_llm:
        disciplines_for_summary = []
        for dn, dr in analysis_results_llm.items():
            if dr.get('nota_necessaria_pf') not in ["N/A", "N/A (Sem PF)", "Erro LLM"] :
                 disciplines_for_summary.append({
                     "disciplina": dn,
                     "nota_necessaria_pf": dr.get('nota_necessaria_pf'),
                     "situacao_pf": dr.get('situacao_pf')
                 })

        if disciplines_for_summary:
            prompt_summary = f"""
            Você é um mentor educacional para estudantes do ensino médio, como um "irmão mais velho" experiente, paciente e compreensivo.
            Seu papel é acolher, orientar e acompanhar o aluno nos estudos. O aluno se chama {student_data.get('nome', 'estudante')}.

            Com base nos seguintes resultados de análise de disciplinas, onde é mostrada a nota necessária na Prova Final (PF) para atingir a média:
            {json.dumps(disciplines_for_summary, ensure_ascii=False, indent=2)}

            Produza um resumo conciso, em texto simples (não JSON), priorizando as disciplinas que exigem maior esforço ou atenção na PF.
            **Use um tom encorajador e de apoio, como se estivesse conversando com {student_data.get('nome', 'estudante')}.**
            Ordene as disciplinas da mais crítica/difícil para a mais tranquila em termos de nota na PF.
            Destaque as 3 a 5 mais importantes e explique brevemente porquê (ex: "Em Matemática, parece que a gente vai precisar de um gás extra na PF, mas tô aqui pra ajudar!").
            Se alguma for impossível (nota > 10 na PF ou string indicando impossibilidade), mencione isso com cuidado e ofereça alternativas ou apoio (ex: "Olha, em Química a conta pra PF ficou bem alta, o que significa que talvez não dê pra alcançar só com ela. Mas não se preocupe, vamos ver o que mais podemos fazer, talvez focar em recuperar outras notas ou conversar com o professor.").
            Se alguma já estiver aprovada independente da PF (situação "Já Aprovado"), parabenize! (ex: "E aí, {student_data.get('nome', 'estudante')}, boa notícia! Em História você já está mandando super bem, parece que já garantiu a aprovação! Show!")
            """
            try:
                print("\nGerando resumo priorizado das disciplinas com LLM...")
                response_summary = GEMINI_MODEL.generate_content(prompt_summary)
                student_data['resumo_priorizado_llm'] = response_summary.text
                print("\n📋 Resumo Priorizado das Disciplinas (LLM):")
                # MODIFICAÇÃO AQUI:
                print(textwrap.fill(response_summary.text, width=100))
            except Exception as e:
                print(f"Erro ao gerar resumo priorizado com LLM: {e}")
                student_data['resumo_priorizado_llm'] = "Não foi possível gerar o resumo priorizado."
        else:
            print("\nNenhuma disciplina com dados suficientes para gerar resumo priorizado de PF.")
            student_data['resumo_priorizado_llm'] = "Sem dados para resumo."


    save_student_data(student_id, student_data)
    return analysis_results_llm


# --- Módulo 5: Geração de Plano de Estudos com LLM ---
def parse_time_range(time_range_str):
    match = re.fullmatch(r"(\d{1,2}:\d{2})-(\d{1,2}:\d{2})", time_range_str.strip())
    if not match:
        return None, None, 0

    start_str, end_str = match.groups()
    try:
        start_time_obj = datetime.strptime(start_str, "%H:%M").time()
        end_time_obj = datetime.strptime(end_str, "%H:%M").time()

        today = datetime.today().date()
        start_dt = datetime.combine(today, start_time_obj)
        end_dt = datetime.combine(today, end_time_obj)

        if end_dt <= start_dt:
            end_dt += timedelta(days=1) # Assume que cruza a meia-noite

        duration = end_dt - start_dt
        duration_hours = duration.total_seconds() / 3600
        if duration_hours <= 0:
             return None, None, 0
        return start_time_obj, end_time_obj, duration_hours
    except ValueError:
        return None, None, 0

def get_study_availability():
    print("\n🗓️ Coleta de Disponibilidade Semanal para Estudos Extras")
    print("   Por favor, informe os períodos de estudo para cada dia.")
    print("   Use o formato HH:MM-HH:MM (ex: 18:00-20:30).")
    print("   Se tiver mais de um período no mesmo dia, separe por vírgula (ex: 09:00-11:00, 14:00-16:00).")
    print("   Deixe em branco se não houver disponibilidade no dia.")

    availability_details = {}
    total_hours_study = 0
    days_of_week = ["Segunda", "Terça", "Quarta", "Quinta", "Sexta", "Sábado", "Domingo"]

    for day in days_of_week:
        availability_details[day] = []
        while True:
            input_str = input(f"  Disponibilidade na {day}: ").strip()
            if not input_str:
                break

            period_strings = [p.strip() for p in input_str.split(',')]
            valid_periods_for_day_input = []
            all_periods_valid_for_day = True

            for period_str in period_strings:
                start_t, end_t, duration_h = parse_time_range(period_str)
                if duration_h > 0:
                    valid_periods_for_day_input.append({
                        "inicio_str": start_t.strftime("%H:%M"),
                        "fim_str": end_t.strftime("%H:%M"),
                        "duracao_horas": duration_h
                    })
                else:
                    print(f"    Formato inválido ('{period_str}') ou duração zero/negativa. Use HH:MM-HH:MM com fim após início.")
                    all_periods_valid_for_day = False
                    break

            if all_periods_valid_for_day:
                availability_details[day].extend(valid_periods_for_day_input)
                break
            else:
                print(f"    Por favor, reinsira a disponibilidade para {day}.")
                availability_details[day] = []

    for day_name, periods in availability_details.items():
        for period in periods:
            total_hours_study += period['duracao_horas']

    if total_hours_study == 0:
        print("\nVocê não indicou nenhuma disponibilidade. Não será possível gerar um plano de estudos detalhado.")
        return None

    print(f"\nTotal de horas de estudo semanais disponíveis: {total_hours_study:.1f} horas.")
    return availability_details


def generate_llm_detailed_study_plan(student_data, analysis_results, availability_details):
    if not GEMINI_MODEL:
        print("LLM não configurada. Não é possível gerar o plano de estudos detalhado.")
        return None
    if not availability_details or sum(sum(p['duracao_horas'] for p in periods) for periods in availability_details.values() if periods) == 0:
        print("Disponibilidade de estudo não fornecida ou zerada. Plano não gerado.")
        return None
    if not analysis_results:
        print("Análise acadêmica não disponível. Plano não gerado.")
        return None

    disciplinas_para_plano = []
    for disc_nome, result in analysis_results.items():
        nota_pf = result.get('nota_necessaria_pf')
        situacao = result.get('situacao_pf', 'Indefinida')

        if isinstance(nota_pf, (float, int)) or (isinstance(nota_pf, str) and "impossível" not in nota_pf.lower() and "n/a" not in nota_pf.lower()):
            if situacao not in ["Já Aprovado", "Confortável", "N/A (Sem PF)"]:
                 disciplinas_para_plano.append({
                     "nome": disc_nome,
                     "nota_necessaria_pf": nota_pf,
                     "situacao_pf": situacao,
                     "bncc_topics": get_bncc_topics(student_data.get('serie'), disc_nome)
                 })
        elif isinstance(nota_pf, str) and "impossível" in nota_pf.lower():
            disciplinas_para_plano.append({
                     "nome": disc_nome,
                     "nota_necessaria_pf": nota_pf,
                     "situacao_pf": situacao,
                     "bncc_topics": get_bncc_topics(student_data.get('serie'), disc_nome)
                 })


    if not disciplinas_para_plano:
        print("Nenhuma disciplina parece precisar de foco para Prova Final no momento ou os dados são insuficientes. Plano não gerado.")
        return None

    def sort_key_disciplinas(d):
        nota = d['nota_necessaria_pf']
        if isinstance(nota, str):
            if "impossível" in nota.lower() or ('>' in nota and re.search(r"[-+]?\d*\.\d+|\d+", nota) and float(re.findall(r"[-+]?\d*\.\d+|\d+", nota)[0]) > 10):
                return float('-inf')
            return 0
        if nota is None: return float('inf')
        return -float(nota)

    disciplinas_para_plano.sort(key=sort_key_disciplinas)

    prompt_availability = {}
    days_of_week_map = {"Segunda": "Segunda-feira", "Terça": "Terça-feira", "Quarta": "Quarta-feira", "Quinta": "Quinta-feira", "Sexta": "Sexta-feira", "Sábado": "Sábado", "Domingo": "Domingo"}
    for day, periods in availability_details.items():
        day_key = days_of_week_map.get(day, day) # Usa o nome completo do dia da semana
        if periods:
            prompt_availability[day_key] = [f"{p['inicio_str']}-{p['fim_str']}" for p in periods]


    prompt_plan = f"""
    Você é um planejador de estudos especialista para alunos do Ensino Médio, com um tom de "irmão mais velho" experiente e compreensivo.
    O aluno se chama {student_data.get('nome', 'estudante')}.

    Dados do aluno:
    - Série: {student_data.get('serie')}
    - Disponibilidade semanal para estudos (períodos específicos por dia):
      {json.dumps(prompt_availability, ensure_ascii=False, indent=2)}
    - Disciplinas prioritárias (com nota necessária na Prova Final e tópicos BNCC relevantes):
      {json.dumps(disciplinas_para_plano, ensure_ascii=False, indent=2)}

    Tarefa:
    Crie um plano de estudos semanal DETALHADO para o aluno, {student_data.get('nome', 'estudante')}.
    1.  **Utilize APENAS os períodos de estudo informados pelo aluno.** Distribua as disciplinas prioritárias dentro desses horários.
    2.  Dê mais tempo para as disciplinas mais críticas (maior nota necessária na PF ou situação 'Crítica'/'Atenção Elevada'/'Impossível').
    3.  Para cada bloco de estudo, especifique:
        - 'dia_semana': (ex: "Segunda-feira") - Use os nomes dos dias como fornecidos na disponibilidade (Segunda-feira, Terça-feira, etc.).
        - 'horario_inicio': (ex: "18:00")
        - 'horario_fim': (ex: "19:30")
        - 'disciplina': Nome da disciplina. Se for uma pausa ou refeição, pode ser "Pausa" ou "Refeição".
        - 'atividade_sugerida': Uma atividade específica e motivadora (ex: "Bora detonar em [Tópico BNCC XYZ]!", "Que tal resolver uns desafios de [assunto] pra ficar craque?", "Vamos fazer um esquenta pra PF com uns exercícios tipo simulado?", "Uma leitura rápida e focada do resumo do capítulo X pra clarear as ideias."). Para pausas, "Pequena pausa para esticar as pernas e tomar uma água!" ou "Hora do lanche!". Seja criativo, variado e use um tom encorajador. Se houver tópicos BNCC, tente usá-los.
        - 'tipo_bloco': "Estudo" ou "Pausa" ou "Refeição".
    4.  Incorpore pausas curtas (10-15 min) a cada 50-90 minutos de estudo DENTRO dos períodos disponíveis. Se um período for curto (ex: 1h), talvez uma pausa não seja necessária ou pode ser ao final.
    5.  Se os blocos de estudo se estenderem por horários típicos de refeição (ex: 12:00-14:00, 19:00-20:30) E o aluno tiver disponibilidade nesses horários, sugira uma pausa para refeição.
    6.  O plano deve ser realista e ajudar {student_data.get('nome', 'estudante')} a se preparar para as Provas Finais.
    7.  **Não crie blocos de estudo fora dos horários de disponibilidade informados.** Certifique-se que horario_fim de um bloco não exceda o fim do período de disponibilidade do dia.

    Retorne o plano APENAS como um array JSON de objetos. Cada objeto é um bloco no plano. Exemplo de um bloco:
    {{ "dia_semana": "Segunda-feira", "horario_inicio": "18:00", "horario_fim": "18:50", "disciplina": "Matemática", "atividade_sugerida": "Revisar Teorema de Pitágoras (EM13MAT101)", "tipo_bloco": "Estudo" }}
    NÃO inclua nenhum texto antes ou depois do array JSON.
    """

    try:
        print("\nGerando plano de estudos detalhado com LLM (isso pode levar um momento)...")
        response_plan = GEMINI_MODEL.generate_content(prompt_plan)
        plan_json = extract_json_from_llm_response(response_plan.text)

        if plan_json and isinstance(plan_json, list) and len(plan_json) > 0:
            print("\n📅 Plano de Estudos Semanal Detalhado (Sugestão da LLM):")
            df_plano = pd.DataFrame(plan_json)
            if not df_plano.empty:
                days_order_ics = ["Segunda-feira", "Terça-feira", "Quarta-feira", "Quinta-feira", "Sexta-feira", "Sábado", "Domingo"]
                try:
                    # Garantir que 'dia_semana' exista antes de tentar categorizar
                    if 'dia_semana' in df_plano.columns and 'horario_inicio' in df_plano.columns:
                        df_plano['dia_semana_cat'] = pd.Categorical(df_plano['dia_semana'], categories=days_order_ics, ordered=True)
                        df_plano.sort_values(by=['dia_semana_cat', 'horario_inicio'], inplace=True)
                        df_plano.drop(columns=['dia_semana_cat'], inplace=True)
                    elif 'dia_semana' in df_plano.columns: # Ordenar apenas por dia se horário não existir
                         df_plano['dia_semana_cat'] = pd.Categorical(df_plano['dia_semana'], categories=days_order_ics, ordered=True)
                         df_plano.sort_values(by=['dia_semana_cat'], inplace=True)
                         df_plano.drop(columns=['dia_semana_cat'], inplace=True)
                except KeyError:
                    print("Aviso: Colunas esperadas ('dia_semana', 'horario_inicio') podem não estar completas no plano da LLM para ordenação.")
                except Exception as e_sort:
                    print(f"Aviso: não foi possível ordenar o plano detalhadamente ({e_sort}).")

                print(df_plano.to_string(index=False))

                export_choice = input("\nDeseja exportar este plano? (xlsx/ics/N): ").strip().lower()
                student_file_prefix = student_data.get('id', 'aluno')
                if export_choice == 'xlsx':
                    filename_xlsx = f"{student_file_prefix}_plano_estudos_detalhado.xlsx"
                    try:
                        df_plano.to_excel(filename_xlsx, index=False)
                        files.download(filename_xlsx)
                        print(f"Plano exportado: '{filename_xlsx}'")
                    except Exception as e: print(f"Erro ao exportar para XLSX: {e}")
                elif export_choice == 'ics':
                    filename_ics = f"{student_file_prefix}_plano_estudos_detalhado.ics"
                    cal = Calendar()
                    today_date = datetime.now().date()

                    days_map_python = {
                        "segunda-feira": 0, "terça-feira": 1, "quarta-feira": 2,
                        "quinta-feira": 3, "sexta-feira": 4, "sábado": 5, "domingo": 6
                    }

                    for _, tarefa in df_plano.iterrows():
                        try:
                            dia_str_tarefa = tarefa.get('dia_semana','').strip().lower()
                            target_weekday_idx = days_map_python.get(dia_str_tarefa)

                            if target_weekday_idx is None:
                                print(f"Aviso: Dia da semana '{tarefa.get('dia_semana')}' não reconhecido para evento ICS. Pulando.")
                                continue

                            current_weekday = today_date.weekday()
                            days_diff = target_weekday_idx - current_weekday
                            if days_diff < 0: days_diff += 7
                            event_date = today_date + timedelta(days=days_diff)

                            start_time_str = tarefa.get('horario_inicio', '09:00')
                            end_time_str = tarefa.get('horario_fim', '10:00')

                            start_h, start_m = map(int, start_time_str.split(':'))
                            end_h, end_m = map(int, end_time_str.split(':'))

                            begin_dt = datetime.combine(event_date, time(start_h, start_m))
                            end_dt = datetime.combine(event_date, time(end_h, end_m))

                            if end_dt <= begin_dt:
                                end_dt = begin_dt + timedelta(hours=1)

                            evt_name = f"{tarefa.get('tipo_bloco', 'Evento').title()}: {tarefa.get('disciplina', '')}"
                            if tarefa.get('tipo_bloco', '').lower() == "estudo":
                                evt_name = f"Estudar: {tarefa.get('disciplina', 'Não especificado')}"
                            elif tarefa.get('tipo_bloco', '').lower() == "pausa":
                                evt_name = f"Pausa: {tarefa.get('atividade_sugerida', 'Descanso')}"
                            elif tarefa.get('tipo_bloco', '').lower() == "refeição":
                                evt_name = f"Refeição: {tarefa.get('atividade_sugerida', 'Alimentação')}"

                            event_obj = Event(name=evt_name,
                                          description=tarefa.get('atividade_sugerida', ''),
                                          begin=begin_dt,
                                          end=end_dt)
                            cal.events.add(event_obj)
                        except Exception as e_ics_event:
                            print(f"Erro ao criar evento ICS para tarefa '{tarefa.get('disciplina')}': {e_ics_event}")
                            print(f"Dados da tarefa: {tarefa.to_dict()}")
                    try:
                        with open(filename_ics, 'w', encoding='utf-8') as f: f.writelines(cal.serialize_iter())
                        files.download(filename_ics)
                        print(f"Plano exportado como '{filename_ics}'")
                    except Exception as e: print(f"Erro ao exportar para ICS: {e}")
            else:
                print("A LLM não retornou um plano de estudos válido em formato de tabela ou a lista estava vazia.")
            return df_plano
        else:
            print("A LLM não retornou um JSON válido ou a lista estava vazia para o plano de estudos.")
            print("Resposta da LLM (início):", response_plan.text[:500] if response_plan.text else "Vazio")
            return None

    except Exception as e:
        print(f"Erro ao gerar plano de estudos com LLM: {e}")
        return None
    return None

# --- Módulo 6: Encorajamento e Próximos Passos ---
def final_encouragement(student_data, analysis_results):
    print("\n💬 Passo de Encorajamento Realista e Próximos Passos")
    nome_aluno = student_data.get('nome', 'Estudante')

    if not analysis_results:
        print(f"Olá, {nome_aluno}! Lembre-se de registrar suas notas e passar pela análise para podermos traçar um plano mais eficaz. Estou aqui para ajudar!")
        return

    resumo_priorizado = student_data.get('resumo_priorizado_llm')
    if resumo_priorizado and resumo_priorizado not in ["Não foi possível gerar o resumo priorizado.", "Sem dados para resumo."]:
        print(f"\nEntão, {nome_aluno}, pra fechar nossa conversa de hoje, aqui vai um resumo do que a gente viu:")
        # MODIFICAÇÃO AQUI:
        print(textwrap.fill(resumo_priorizado, width=100))
        print(f"\nLembre-se, {nome_aluno}, este é um guia pra te ajudar a organizar os estudos. O mais importante é você encontrar o que funciona melhor pra você e não desistir, tá bom? Qualquer coisa, é só me chamar!")
        print("Com foco e sua dedicação, você vai longe! Acredito em você! 💪 Bora pra cima!")
    else:
        situacao_geral, disc_exemplo = "Confortável/OK", None
        for disc_nome, result in analysis_results.items():
            sit_pf = result.get('situacao_pf', 'Indefinida')
            if sit_pf == "Crítica": situacao_geral, disc_exemplo = "Crítica", disc_nome; break
            if sit_pf in ["Atenção Elevada", "Atenção"] and situacao_geral != "Crítica":
                situacao_geral, disc_exemplo = "Atenção", disc_nome

        fallback_message = ""
        if situacao_geral == "Crítica":
            msg_disciplina = f" principalmente em {disc_exemplo}" if disc_exemplo else " em algumas matérias"
            fallback_message = (f"Olha, {nome_aluno}, sei que a situação parece um pouco apertada{msg_disciplina}, mas não é hora de desanimar, viu?\n"
                                f"Pensa que cada esforcinho agora faz uma diferença enorme lá na frente! 💪 Com o plano que a gente montou e seu foco, dá pra virar o jogo. Tô aqui na torcida e pra te ajudar no que precisar. Vamos juntos nessa!")
        elif situacao_geral == "Atenção":
            msg_disciplina = f", como em {disc_exemplo}," if disc_exemplo else ""
            fallback_message = (f"Aí sim, {nome_aluno}! Você tá no caminho certo, mandando bem! Algumas matérias{msg_disciplina} vão pedir um pouquinho mais da sua atenção agora, mas é super normal.\n"
                                f"Com o plano e sua dedicação, tenho certeza que você vai tirar de letra. Mantenha esse ritmo bom! Se precisar ajustar alguma coisa ou tiver alguma dúvida, é só me chamar, combinado? 😉")
        else:
            fallback_message = (f"Parabéns, {nome_aluno}! Pelo que vi aqui, sua situação acadêmica está bem encaminhada ou não tivemos todos os detalhes para uma análise super profunda! 🥳\n"
                                f"Se a gente fez um plano, é pra continuar nesse pique ou dar aquele gás final. Continue assim que é sucesso! E se algo mudar, me atualiza!")

        # MODIFICAÇÃO AQUI:
        print(textwrap.fill(fallback_message, width=100))


    print("\nLembre-se de voltar para atualizarmos seu progresso! Até a próxima! 😊")


# --- Fluxo Principal de Execução ---
if __name__ == "__main__":
    mount_drive()

    student_id, student_data = collect_initial_info()

    student_data = collect_academic_info(student_id, student_data)

    analysis_results_llm = {}
    if student_data.get('disciplinas'):
        if any(d.get('media_minima') is not None for d_name, d in student_data['disciplinas'].items()):
            analysis_results_llm = analyze_academic_situation(student_id, student_data)
        else:
            print("\nAnálise acadêmica não realizada: nenhuma disciplina possui média mínima definida. Por favor, valide os dados.")
    else:
        print("\nNenhuma informação de disciplina foi coletada para análise.")

    study_plan_df = None
    if analysis_results_llm and GEMINI_MODEL:
        needs_planning = False
        for res_name, res_data in analysis_results_llm.items():
            sit_pf = res_data.get('situacao_pf', 'Indefinida')
            nota_pf = res_data.get('nota_necessaria_pf')
            if sit_pf not in ["Já Aprovado", "N/A (Sem PF)", "Indefinida"] or \
               (isinstance(nota_pf, str) and "impossível" in nota_pf.lower()):
                needs_planning = True
                break

        if needs_planning:
            availability = get_study_availability()
            if availability:
                study_plan_df = generate_llm_detailed_study_plan(student_data, analysis_results_llm, availability)
            else:
                print("\nPlano de estudos não gerado pois não foi informada disponibilidade semanal.")
        else:
            print("\nPlano de estudos não gerado: todas as disciplinas parecem estar em situação confortável, já aprovadas, ou não necessitam de cálculo de Prova Final.")

    elif not GEMINI_MODEL and student_data.get('disciplinas'):
        print("\nPlano de estudos detalhado não pode ser gerado pois a API da LLM não está configurada.")
    elif not student_data.get('disciplinas'):
        print("\nPlano de estudos não pôde ser gerado pois não há dados de disciplinas.")


    final_encouragement(student_data, analysis_results_llm)

    print("\n--- Sessão Concluída ---")

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.1/40.1 kB 769.3 kB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.6/232.6 kB 7.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.4/66.4 kB 2.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 80.2/80.2 kB 5.1 MB/s eta 0:00:00
API do Google Gemini configurada com sucesso.
Mounted at /content/drive
Google Drive montado com sucesso.
🎓 Bem-vindo(a) ao Mentor Educacional Personalizado! 😊
Para começarmos, qual é o seu primeiro nome (usaremos como identificador)? AL

Olá, AL! Parece que é nossa primeira conversa.
Qual série do Ensino Médio você está cursando (1ª Série, 2ª Série ou 3ª Série)? 2ª Série
Ótimo, AL da 2ª Série!
Dados do aluno AL salvos em /content/drive/MyDrive/DadosAlunosColab/AL.json

🗂️ Passo 2 – Coleta/Atualização de Informações Acadêmicas
Para uma análise precisa, preciso das suas notas e da estrutura de avaliação.

Escolha uma opção:
  A - Preencher/Editar Manualmente as informações das

Saving BOLETIM-TESTE-1.pdf to BOLETIM-TESTE-1.pdf
Arquivo "BOLETIM-TESTE-1.pdf" enviado.

Enviando texto do PDF para análise pela LLM (isso pode levar um momento)...

Resposta da LLM recebida.
Dados JSON extraídos e analisados com sucesso pela LLM.

--- Dados Extraídos do PDF pela LLM ---

Disciplina: Biologia
  Média Mínima: 6.0
  Fórmula: 0.10 × EN + 0.40 × AV + 0.50 × PF
  Avaliações:
    - Nome: Engajamento, Peso: 10.0%, Nota: 5.00, Faltante: False
    - Nome: Avaliatório, Peso: 40.0%, Nota: 4.85, Faltante: False
    - Nome: Prova Final (Prova Final), Peso: 50.0%, Nota: N/A, Faltante: True

Disciplina: História
  Média Mínima: 6.0
  Fórmula: 0.10 × EN + 0.40 × AV + 0.50 × PF
  Avaliações:
    - Nome: Engajamento, Peso: 10.0%, Nota: 5.00, Faltante: False
    - Nome: Avaliatório, Peso: 40.0%, Nota: 7.75, Faltante: False
    - Nome: Prova Final (Prova Final), Peso: 50.0%, Nota: N/A, Faltante: True

Disciplina: Inglês
  Média Mínima: 6.0
  Fórmula: 0.10 × EN + 0.40 × AV + 0.50 × PF
  A

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

Plano exportado: 'AL_plano_estudos_detalhado.xlsx'

💬 Passo de Encorajamento Realista e Próximos Passos

Então, AL, pra fechar nossa conversa de hoje, aqui vai um resumo do que a gente viu:
E aí, AL, tudo bem?  Vamos dar uma olhada nessas notas da prova final, sem pressão!  A gente
consegue resolver isso juntos.  Olha só, analisando os resultados, algumas disciplinas pedem um
pouco mais de atenção, beleza? Vamos focar nas mais importantes, tá?  1. **Língua Portuguesa
(7.40):**  Essa precisa de um foco extra, AL.  7.40 é uma nota alta na PF, mas com dedicação, a
gente chega lá! Vamos planejar um cronograma de estudos, focar nos pontos que você sente mais
dificuldade e praticar bastante.  2. **Biologia (7.30):**  Semelhante a Português, essa também exige
um esforço considerável na PF.  Vamos revisar os assuntos mais complexos, talvez com exercícios
específicos e revisão de provas anteriores?  3. **Inglês (7.25):**  Essa também está pertinho, AL!
Com um bom empenho na PF, você garante ess